In [5]:
import plotly.graph_objects as go

# 노드 정의 (이름과 좌표 x, y)
# particle(0,2) / gas(-1,1), pressure(0,1), temp(1,1) / h2(-1.5,0), nf3(-0.5,0)
x_nodes = [0, -1, 0, 1, -1.5, -0.5]
y_nodes = [2, 1, 1, 1, 0, 0]
node_names = ["particle", "gas", "pressure", "temp", "h2", "nf3"]

# 엣지(선) 정의 (연결할 좌표 쌍)
# (particle -> gas), (particle -> pressure), (particle -> temp)
# (gas -> h2), (gas -> nf3)
x_edges = [0, -1, None, 0, 0, None, 0, 1, None, -1, -1.5, None, -1, -0.5]
y_edges = [2, 1, None, 2, 1, None, 2, 1, None, 1, 0, None, 1, 0]

fig = go.Figure()

# 선 그리기
fig.add_trace(go.Scatter(
    x=x_edges, y=y_edges,
    mode='lines',
    line=dict(color='black', width=1),
    hoverinfo='none'
))

# 점(노드) 그리기
fig.add_trace(go.Scatter(
    x=x_nodes, y=y_nodes,
    mode='markers+text',
    text=node_names,
    textposition="bottom center",
    marker=dict(size=20, color='skyblue', line=dict(color='black', width=1)),
    hoverinfo='text'
))

fig.update_layout(
    title="Particle Tree Diagram",
    showlegend=False,
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
)

fig.show()

In [6]:
fig.write_html('test.html')

In [19]:
import plotly.graph_objects as go
import pandas as pd 
data = pd.DataFrame()
data['sensor'] = ['top temp', 'bottom temp', 'h2', 'nf3', 'pressure', 'throttle valve']
data.index=data['sensor']
data['group']= 'temp'
data.loc['h2','group'] = 'gas'
data.loc['nf3','group'] = 'gas'
data.loc['pressure','group'] = 'press'
data.loc['throttle valve','group'] = 'press'
data.to_csv('data.csv')

In [8]:

labels = ["particle", "gas", "pressure", "temp", "h2", "nf3"]
parents = ["", "particle", "particle", "particle", "gas", "gas"]

fig = go.Figure(go.Sunburst(
    labels=labels,
    parents=parents,
    branchvalues="total"  # 하위 항목의 크기 합이 부모와 같게 (데이터가 있을 경우 유용)
))

fig.update_layout(
    title="Particle Hierarchy Structure (Sunburst)",
    margin=dict(t=50, l=25, r=25, b=25)
)

fig.write_html('test.html')

In [7]:
import pandas as pd
import json
import random
import plotly.graph_objects as go

# ---------------------------------------------------------
# 1. 데이터 정의 (Pandas)
# ---------------------------------------------------------
data = pd.DataFrame()
data['sensor'] = ['top temp', 'bottom temp', 'h2', 'nf3', 'pressure', 'throttle valve']
data.index = data['sensor']
data['group'] = 'temp'
data.loc['h2', 'group'] = 'gas'
data.loc['nf3', 'group'] = 'gas'
data.loc['pressure', 'group'] = 'press'
data.loc['throttle valve', 'group'] = 'press'

# 시계열 데이터 생성
def generate_history():
    base = random.randint(30, 80)
    return [base + random.randint(-5, 5) for _ in range(20)]

data['history'] = [generate_history() for _ in range(len(data))]
data['value'] = [val[-1] for val in data['history']]

# ---------------------------------------------------------
# 2. Python에서 Plotly 차트 설정 생성
# ---------------------------------------------------------
def create_plotly_config(name, y_values):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        y=y_values,
        mode='lines+markers',
        name=name,
        line=dict(color='#d32f2f', width=3, shape='spline'),
        marker=dict(size=8, color='white', line=dict(width=2, color='#d32f2f'))
    ))
    fig.update_layout(
        title=dict(text=f"{name} Trend", font=dict(size=20, color='#333')),
        xaxis=dict(title='Time Step', showgrid=True, gridcolor='#eee'),
        yaxis=dict(title='Value', showgrid=True, gridcolor='#eee'),
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        margin=dict(l=50, r=30, t=60, b=50),
        showlegend=False
    )
    return json.loads(fig.to_json())

# ---------------------------------------------------------
# 3. JSON 구조 변환
# ---------------------------------------------------------
groups = data.groupby('group')
children_list = []

for group_name, group_df in groups:
    sensors = []
    for _, row in group_df.iterrows():
        chart_config = create_plotly_config(row['sensor'], row['history'])
        sensors.append({
            "name": row['sensor'],
            "value": row['value'],
            "chart_config": chart_config,
            "type": "sensor"
        })
    children_list.append({
        "name": group_name,
        "children": sensors,
        "type": "group"
    })

tree_structure = {
    "name": "Particle Equipment",
    "children": children_list,
    "type": "root"
}

json_data = json.dumps(tree_structure)

# ---------------------------------------------------------
# 4. HTML/JS 생성 (Highlight Logic 추가됨)
# ---------------------------------------------------------
html_content = f"""
<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">
    <title>Tree with Path Highlighting</title>
    <script src="https://d3js.org/d3.v7.min.js"></script>
    <script src="https://cdn.plot.ly/plotly-2.27.0.min.js"></script>
    
    <style>
        body {{
            font-family: 'Segoe UI', sans-serif;
            background-color: #f4f6f9;
            margin: 0; padding: 20px;
            display: flex; flex-direction: column; height: 100vh; box-sizing: border-box;
        }}
        
        .header {{ 
            margin-bottom: 10px; padding: 10px 20px; 
            background: white; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); 
            display: flex; justify-content: space-between; align-items: center;
        }}
        
        .main-container {{ display: flex; flex: 1; gap: 20px; overflow: hidden; }}
        
        #tree-wrapper {{ flex: 1; background: white; border-radius: 12px; border: 1px solid #e0e0e0; position: relative; }}
        #chart-wrapper {{ flex: 1; background: white; border-radius: 12px; border: 1px solid #e0e0e0; padding: 10px; display: flex; align-items: center; justify-content: center; }}
        
        /* --- D3 스타일 --- */
        .node circle {{ 
            fill: #fff; stroke: steelblue; stroke-width: 3px; 
            cursor: pointer; transition: all 0.3s; 
        }}
        .node circle:hover {{ fill: #e3f2fd; stroke-width: 5px; }}
        
        .node text {{ font: 12px sans-serif; font-weight: bold; pointer-events: none; }}
        
        /* 기본 링크 스타일 */
        .link {{ 
            fill: none; stroke: #ccc; stroke-width: 2px; 
            transition: stroke 0.4s, stroke-width 0.4s, opacity 0.4s; 
        }}
        
        /* --- 하이라이트 스타일 (빨간색 경로) --- */
        .link.highlight {{ 
            stroke: #d32f2f; 
            stroke-width: 4px; 
            stroke-opacity: 1;
        }}
        
        /* 선택되지 않은 경로는 흐리게 */
        .link.dimmed {{ 
            stroke-opacity: 0.2; 
        }}
        
        /* 선택된 노드 */
        .node circle.selected {{ 
            fill: #ffcdd2; stroke: #d32f2f; stroke-width: 5px; 
        }}
        
        .placeholder {{ color: #999; font-size: 1.2rem; }}
    </style>
</head>
<body>

<div class="header">
    <strong>Sensor Tree (Click node to highlight path)</strong>
    <button onclick="resetView()" style="padding:5px 10px; cursor:pointer;">Reset View</button>
</div>

<div class="main-container">
    <div id="tree-wrapper"></div>
    <div id="chart-wrapper">
        <div id="plotly-chart" style="width:100%; height:100%;"></div>
        <div id="placeholder" class="placeholder">Select a sensor node</div>
    </div>
</div>

<script>
    const treeData = {json_data};

    // 캔버스 설정
    const container = document.getElementById("tree-wrapper");
    const width = container.clientWidth;
    const height = container.clientHeight;
    
    // Zoom 설정
    const zoom = d3.zoom().on("zoom", (e) => g.attr("transform", e.transform));
    const svg = d3.select("#tree-wrapper").append("svg")
        .attr("width", "100%").attr("height", "100%")
        .call(zoom)
        .on("dblclick.zoom", null);
    
    const g = svg.append("g");
    
    // 트리 데이터 처리
    const root = d3.hierarchy(treeData, d => d.children);
    const tree = d3.tree().nodeSize([50, 160]); // [높이 간격, 너비 간격]
    const nodes = tree(root);

    // --- 링크(Edge) 그리기 ---
    const link = g.selectAll(".link")
        .data(nodes.links())
        .enter().append("path")
        .attr("class", "link")
        .attr("d", d3.linkHorizontal().x(d => d.y).y(d => d.x));

    // --- 노드(Node) 그리기 ---
    const node = g.selectAll(".node")
        .data(nodes.descendants())
        .enter().append("g")
        .attr("class", "node")
        .attr("transform", d => "translate(" + d.y + "," + d.x + ")");

    // 노드 원 + 클릭 이벤트
    node.append("circle")
        .attr("r", 10)
        .on("click", function(event, d) {{
            
            // ----------------------------------------------------
            // [핵심 로직] 클릭 시 경로 하이라이트 & 차트 그리기
            // ----------------------------------------------------
            
            // 1. 선택된 노드 스타일 적용
            d3.selectAll("circle").classed("selected", false); // 초기화
            d3.select(this).classed("selected", true);         // 현재 노드 선택
            
            // 2. 경로 계산 (현재 노드부터 루트까지의 조상들)
            const ancestors = new Set(d.ancestors());

            // 3. 링크 스타일 업데이트
            // 링크의 target(자식 쪽 노드)이 조상 목록(ancestors)에 있으면 경로에 포함된 것임
            link.classed("highlight", l => ancestors.has(l.target))
                .classed("dimmed", l => !ancestors.has(l.target));

            // 4. 차트 그리기 (Plotly)
            if (d.data.chart_config) {{
                document.getElementById("placeholder").style.display = "none";
                const config = d.data.chart_config;
                Plotly.newPlot('plotly-chart', config.data, config.layout, {{responsive: true}});
            }} else {{
                // 그룹 노드를 클릭한 경우 차트는 지움 (원하면 유지 가능)
                Plotly.purge('plotly-chart');
                document.getElementById("placeholder").style.display = "block";
            }}
            
            event.stopPropagation(); // 이벤트 버블링 방지
        }});

    // 노드 텍스트
    node.append("text")
        .attr("dy", ".35em")
        .attr("x", d => d.children ? -15 : 15)
        .style("text-anchor", d => d.children ? "end" : "start")
        .text(d => d.data.name);

    // 초기화 함수
    window.resetView = function() {{
        // 스타일 초기화
        link.classed("highlight", false).classed("dimmed", false);
        d3.selectAll("circle").classed("selected", false);
        
        // 줌 초기화
        svg.transition().duration(750).call(
            zoom.transform, 
            d3.zoomIdentity.translate(80, height/2)
        );
        
        // 차트 초기화
        Plotly.purge('plotly-chart');
        document.getElementById("placeholder").style.display = "block";
    }}

    // 시작 시 중앙 정렬
    resetView();

</script>
</body>
</html>
"""

with open("tree_path_highlight.html", "w", encoding="utf-8") as f:
    f.write(html_content)

print("파일 생성 완료: tree_path_highlight.html")
print("이제 노드를 클릭하면 '루트'부터 '해당 노드'까지의 연결선이 빨간색으로 변경됩니다.")

파일 생성 완료: tree_path_highlight.html
이제 노드를 클릭하면 '루트'부터 '해당 노드'까지의 연결선이 빨간색으로 변경됩니다.


In [8]:
import pandas as pd
import json
import random
import plotly.graph_objects as go

# ---------------------------------------------------------
# 1. 데이터 정의 (Pandas)
# ---------------------------------------------------------
data = pd.DataFrame()
data['sensor'] = ['top temp', 'bottom temp', 'h2', 'nf3', 'pressure', 'throttle valve']
data.index = data['sensor']
data['group'] = 'temp'
data.loc['h2', 'group'] = 'gas'
data.loc['nf3', 'group'] = 'gas'
data.loc['pressure', 'group'] = 'press'
data.loc['throttle valve', 'group'] = 'press'

# 시계열 데이터 생성
def generate_history():
    base = random.randint(30, 80)
    return [base + random.randint(-5, 5) for _ in range(20)]

data['history'] = [generate_history() for _ in range(len(data))]
# 현재 값 (시각화에 표시될 값)
data['value'] = [val[-1] for val in data['history']]

# ---------------------------------------------------------
# 2. Python에서 Plotly 차트 설정 생성
# ---------------------------------------------------------
def create_plotly_config(name, y_values):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        y=y_values,
        mode='lines+markers',
        name=name,
        line=dict(color='#d32f2f', width=3, shape='spline'),
        marker=dict(size=8, color='white', line=dict(width=2, color='#d32f2f'))
    ))
    fig.update_layout(
        title=dict(text=f"{name} Trend", font=dict(size=20, color='#333')),
        xaxis=dict(title='Time Step', showgrid=True, gridcolor='#eee'),
        yaxis=dict(title='Value', showgrid=True, gridcolor='#eee'),
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        margin=dict(l=50, r=30, t=60, b=50),
        showlegend=False
    )
    return json.loads(fig.to_json())

# ---------------------------------------------------------
# 3. JSON 구조 변환
# ---------------------------------------------------------
groups = data.groupby('group')
children_list = []

for group_name, group_df in groups:
    sensors = []
    for _, row in group_df.iterrows():
        chart_config = create_plotly_config(row['sensor'], row['history'])
        sensors.append({
            "name": row['sensor'],
            "value": row['value'],
            "chart_config": chart_config,
            "type": "sensor"
        })
    children_list.append({
        "name": group_name,
        "children": sensors,
        "type": "group"
    })

tree_structure = {
    "name": "Particle Equipment",
    "children": children_list,
    "type": "root"
}

json_data = json.dumps(tree_structure)

# ---------------------------------------------------------
# 4. HTML/JS 생성
# ---------------------------------------------------------
html_content = f"""
<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">
    <title>Tree with Value Label</title>
    <script src="https://d3js.org/d3.v7.min.js"></script>
    <script src="https://cdn.plot.ly/plotly-2.27.0.min.js"></script>
    
    <style>
        body {{
            font-family: 'Segoe UI', sans-serif;
            background-color: #f4f6f9;
            margin: 0; padding: 20px;
            display: flex; flex-direction: column; height: 100vh; box-sizing: border-box;
        }}
        
        .header {{ 
            margin-bottom: 10px; padding: 10px 20px; 
            background: white; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); 
            display: flex; justify-content: space-between; align-items: center;
        }}
        
        .main-container {{ display: flex; flex: 1; gap: 20px; overflow: hidden; }}
        
        #tree-wrapper {{ flex: 1; background: white; border-radius: 12px; border: 1px solid #e0e0e0; position: relative; }}
        #chart-wrapper {{ flex: 1; background: white; border-radius: 12px; border: 1px solid #e0e0e0; padding: 10px; display: flex; align-items: center; justify-content: center; }}
        
        /* D3 Elements */
        .node circle {{ fill: #fff; stroke: steelblue; stroke-width: 3px; cursor: pointer; transition: all 0.3s; }}
        .node circle:hover {{ fill: #e3f2fd; stroke-width: 5px; }}
        .node text {{ font: 12px sans-serif; font-weight: bold; pointer-events: none; }}
        
        .link {{ fill: none; stroke: #ccc; stroke-width: 2px; transition: all 0.4s; }}
        
        /* Highlight Styles */
        .link.highlight {{ stroke: #d32f2f; stroke-width: 4px; stroke-opacity: 1; }}
        .link.dimmed {{ stroke-opacity: 0.2; }}
        .node circle.selected {{ fill: #ffcdd2; stroke: #d32f2f; stroke-width: 5px; }}
        
        /* Value Label Style (새로 추가됨) */
        .value-label-bg {{
            fill: #333;
            opacity: 0.9;
        }}
        .value-label-text {{
            font-size: 11px;
            fill: white;
            font-weight: bold;
            pointer-events: none;
        }}

        .placeholder {{ color: #999; font-size: 1.2rem; }}
    </style>
</head>
<body>

<div class="header">
    <strong>Sensor Tree (Click to see Value & Path)</strong>
    <button onclick="resetView()" style="padding:5px 10px; cursor:pointer;">Reset View</button>
</div>

<div class="main-container">
    <div id="tree-wrapper"></div>
    <div id="chart-wrapper">
        <div id="plotly-chart" style="width:100%; height:100%;"></div>
        <div id="placeholder" class="placeholder">Select a sensor node</div>
    </div>
</div>

<script>
    const treeData = {json_data};

    const container = document.getElementById("tree-wrapper");
    const width = container.clientWidth;
    const height = container.clientHeight;
    
    // Zoom
    const zoom = d3.zoom().on("zoom", (e) => g.attr("transform", e.transform));
    const svg = d3.select("#tree-wrapper").append("svg")
        .attr("width", "100%").attr("height", "100%")
        .call(zoom)
        .on("dblclick.zoom", null);
    
    const g = svg.append("g");
    
    // Tree Layout
    const root = d3.hierarchy(treeData, d => d.children);
    const tree = d3.tree().nodeSize([50, 160]);
    const nodes = tree(root);

    // Links
    const link = g.selectAll(".link")
        .data(nodes.links())
        .enter().append("path")
        .attr("class", "link")
        .attr("d", d3.linkHorizontal().x(d => d.y).y(d => d.x));

    // Nodes
    const node = g.selectAll(".node")
        .data(nodes.descendants())
        .enter().append("g")
        .attr("class", "node")
        .attr("transform", d => "translate(" + d.y + "," + d.x + ")");

    node.append("circle")
        .attr("r", 10)
        .on("click", function(event, d) {{
            
            // ----------------------------------------------------
            // 1. 하이라이트 & 차트 로직 (기존)
            // ----------------------------------------------------
            d3.selectAll("circle").classed("selected", false);
            d3.select(this).classed("selected", true);
            
            const ancestors = new Set(d.ancestors());
            link.classed("highlight", l => ancestors.has(l.target))
                .classed("dimmed", l => !ancestors.has(l.target));

            if (d.data.chart_config) {{
                document.getElementById("placeholder").style.display = "none";
                const config = d.data.chart_config;
                Plotly.newPlot('plotly-chart', config.data, config.layout, {{responsive: true}});
            }} else {{
                Plotly.purge('plotly-chart');
                document.getElementById("placeholder").style.display = "block";
            }}
            
            // ----------------------------------------------------
            // 2. [추가됨] 값(Value) 표시 로직
            // ----------------------------------------------------
            
            // 기존에 떠있는 모든 라벨 제거
            d3.selectAll(".value-label-group").remove();

            // 값이 있는 경우에만 라벨 생성
            if (d.data.value !== undefined) {{
                // 현재 클릭된 노드 그룹(g) 선택
                const parentGroup = d3.select(this.parentNode);
                
                // 라벨을 담을 그룹 생성 (위치 조정을 위해)
                const labelGroup = parentGroup.append("g")
                    .attr("class", "value-label-group")
                    .attr("transform", "translate(0, -25)"); // 노드 위로 25px 이동

                // 라벨 배경 (검은색 박스)
                labelGroup.append("rect")
                    .attr("class", "value-label-bg")
                    .attr("x", -25) // 중심 맞추기
                    .attr("y", -10)
                    .attr("width", 50)
                    .attr("height", 20)
                    .attr("rx", 5); // 둥근 모서리

                // 라벨 텍스트
                labelGroup.append("text")
                    .attr("class", "value-label-text")
                    .attr("x", 0)
                    .attr("y", 4) // 텍스트 수직 중앙 정렬 보정
                    .attr("text-anchor", "middle")
                    .text(d.data.value);
            }}

            event.stopPropagation();
        }});

    node.append("text")
        .attr("dy", ".35em")
        .attr("x", d => d.children ? -15 : 15)
        .style("text-anchor", d => d.children ? "end" : "start")
        .text(d => d.data.name);

    window.resetView = function() {{
        link.classed("highlight", false).classed("dimmed", false);
        d3.selectAll("circle").classed("selected", false);
        d3.selectAll(".value-label-group").remove(); // 라벨도 초기화
        
        svg.transition().duration(750).call(
            zoom.transform, 
            d3.zoomIdentity.translate(80, height/2)
        );
        
        Plotly.purge('plotly-chart');
        document.getElementById("placeholder").style.display = "block";
    }}

    resetView();

</script>
</body>
</html>
"""

with open("tree_value_label.html", "w", encoding="utf-8") as f:
    f.write(html_content)

print("파일 생성 완료: tree_value_label.html")
print("노드를 클릭하면 [빨간 경로] + [우측 차트] + [노드 위 Value 라벨]이 동시에 표시됩니다.")

파일 생성 완료: tree_value_label.html
노드를 클릭하면 [빨간 경로] + [우측 차트] + [노드 위 Value 라벨]이 동시에 표시됩니다.


In [1]:
import pandas as pd
import json
import random
import plotly.graph_objects as go

# ---------------------------------------------------------
# 1. 데이터 정의 (Pandas)
# ---------------------------------------------------------
data = pd.DataFrame()
data['sensor'] = ['top temp', 'bottom temp', 'h2', 'nf3', 'pressure', 'throttle valve']
data.index = data['sensor']
data['group'] = 'temp'
data.loc['h2', 'group'] = 'gas'
data.loc['nf3', 'group'] = 'gas'
data.loc['pressure', 'group'] = 'press'
data.loc['throttle valve', 'group'] = 'press'

# 시계열 데이터 생성 함수
def generate_history(base_val=None):
    if base_val is None:
        base = random.randint(30, 80)
    else:
        base = base_val
    return [base + random.randint(-5, 5) for _ in range(20)]

data['history'] = [generate_history() for _ in range(len(data))]
data['value'] = [val[-1] for val in data['history']]

# ---------------------------------------------------------
# 2. Python에서 Plotly 차트 설정 생성 (메인 센서용)
# ---------------------------------------------------------
def create_plotly_config(name, y_values, color='#d32f2f'):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        y=y_values,
        mode='lines+markers',
        name=name,
        line=dict(color=color, width=3, shape='spline'),
        marker=dict(size=8, color='white', line=dict(width=2, color=color))
    ))
    fig.update_layout(
        title=dict(text=f"{name} Trend", font=dict(size=20, color='#333')),
        xaxis=dict(title='Time Step', showgrid=True, gridcolor='#eee'),
        yaxis=dict(title='Value', showgrid=True, gridcolor='#eee'),
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        margin=dict(l=50, r=30, t=60, b=50),
        showlegend=False
    )
    return json.loads(fig.to_json())

# ---------------------------------------------------------
# 3. JSON 구조 변환 (+ 연관 센서 40개 추가)
# ---------------------------------------------------------
groups = data.groupby('group')
children_list = []

for group_name, group_df in groups:
    sensors = []
    for _, row in group_df.iterrows():
        # 메인 센서 차트 설정 (기존 방식 유지)
        chart_config = create_plotly_config(row['sensor'], row['history'])
        
        # [추가됨] 연관 센서 40개 생성
        related_sensors = []
        for i in range(1, 41):
            rel_name = f"{row['sensor']}_sub_{i:02d}" # 예: top temp_sub_01
            rel_history = generate_history(base_val=row['value']) # 메인 값과 비슷하게 생성
            related_sensors.append({
                "name": rel_name,
                "value": rel_history[-1],
                "history": rel_history # 차트 설정을 다 만들면 너무 무거우므로 데이터만 전달
            })

        sensors.append({
            "name": row['sensor'],
            "value": row['value'],
            "chart_config": chart_config,
            "related_sensors": related_sensors, # 여기에 리스트 추가
            "type": "sensor"
        })
    children_list.append({
        "name": group_name,
        "children": sensors,
        "type": "group"
    })

tree_structure = {
    "name": "Particle Equipment",
    "children": children_list,
    "type": "root"
}

json_data = json.dumps(tree_structure)

# ---------------------------------------------------------
# 4. HTML/JS 생성 (3단 레이아웃 적용)
# ---------------------------------------------------------
html_content = f"""
<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">
    <title>Tree with Related Sensors</title>
    <script src="https://d3js.org/d3.v7.min.js"></script>
    <script src="https://cdn.plot.ly/plotly-2.27.0.min.js"></script>
    
    <style>
        body {{
            font-family: 'Segoe UI', sans-serif;
            background-color: #f4f6f9;
            margin: 0; padding: 20px;
            display: flex; flex-direction: column; height: 100vh; box-sizing: border-box;
        }}
        
        .header {{ 
            margin-bottom: 10px; padding: 10px 20px; 
            background: white; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); 
            display: flex; justify-content: space-between; align-items: center;
        }}
        
        /* 3단 레이아웃 컨테이너 */
        .main-container {{ display: flex; flex: 1; gap: 15px; overflow: hidden; }}
        
        #tree-wrapper {{ flex: 2; background: white; border-radius: 12px; border: 1px solid #e0e0e0; position: relative; }}
        
        /* [추가됨] 연관 센서 리스트 영역 */
        #list-wrapper {{ 
            flex: 1; min-width: 200px; background: white; border-radius: 12px; border: 1px solid #e0e0e0; 
            display: flex; flex-direction: column; overflow: hidden;
        }}
        #list-header {{ padding: 15px; border-bottom: 1px solid #eee; font-weight: bold; background: #fafafa; }}
        #list-content {{ flex: 1; overflow-y: auto; padding: 10px; }}
        
        #chart-wrapper {{ flex: 2; background: white; border-radius: 12px; border: 1px solid #e0e0e0; padding: 10px; display: flex; align-items: center; justify-content: center; }}
        
        /* D3 Elements */
        .node circle {{ fill: #fff; stroke: steelblue; stroke-width: 3px; cursor: pointer; transition: all 0.3s; }}
        .node circle:hover {{ fill: #e3f2fd; stroke-width: 5px; }}
        .node text {{ font: 12px sans-serif; font-weight: bold; pointer-events: none; }}
        
        .link {{ fill: none; stroke: #ccc; stroke-width: 2px; transition: all 0.4s; }}
        
        /* Highlight Styles */
        .link.highlight {{ stroke: #d32f2f; stroke-width: 4px; stroke-opacity: 1; }}
        .link.dimmed {{ stroke-opacity: 0.2; }}
        .node circle.selected {{ fill: #ffcdd2; stroke: #d32f2f; stroke-width: 5px; }}
        
        /* Value Label Style */
        .value-label-bg {{ fill: #333; opacity: 0.9; }}
        .value-label-text {{ font-size: 11px; fill: white; font-weight: bold; pointer-events: none; }}

        /* List Item Style */
        .sensor-item {{
            padding: 8px 12px; margin-bottom: 5px; border-radius: 6px; cursor: pointer;
            border: 1px solid #eee; transition: background 0.2s; font-size: 0.9rem;
            display: flex; justify-content: space-between;
        }}
        .sensor-item:hover {{ background-color: #f5f5f5; }}
        .sensor-item.active {{ background-color: #e3f2fd; border-color: #2196f3; color: #1565c0; font-weight: bold; }}
        .sensor-val {{ color: #666; font-size: 0.85rem; }}

        .placeholder {{ color: #999; font-size: 1.2rem; }}
    </style>
</head>
<body>

<div class="header">
    <strong>Detailed Sensor Analysis (Tree -> Related List -> Chart)</strong>
    <button onclick="resetView()" style="padding:5px 10px; cursor:pointer;">Reset View</button>
</div>

<div class="main-container">
    <div id="tree-wrapper"></div>
    
    <!-- 중간 리스트 영역 -->
    <div id="list-wrapper">
        <div id="list-header">Related Sensors</div>
        <div id="list-content">
            <div style="color:#999; text-align:center; margin-top:20px;">Select a node</div>
        </div>
    </div>

    <div id="chart-wrapper">
        <div id="plotly-chart" style="width:100%; height:100%;"></div>
        <div id="placeholder" class="placeholder">Select a sensor</div>
    </div>
</div>

<script>
    const treeData = {json_data};

    const container = document.getElementById("tree-wrapper");
    const width = container.clientWidth;
    const height = container.clientHeight;
    
    const zoom = d3.zoom().on("zoom", (e) => g.attr("transform", e.transform));
    const svg = d3.select("#tree-wrapper").append("svg")
        .attr("width", "100%").attr("height", "100%")
        .call(zoom)
        .on("dblclick.zoom", null);
    
    const g = svg.append("g");
    
    const root = d3.hierarchy(treeData, d => d.children);
    const tree = d3.tree().nodeSize([50, 160]);
    const nodes = tree(root);

    const link = g.selectAll(".link")
        .data(nodes.links())
        .enter().append("path")
        .attr("class", "link")
        .attr("d", d3.linkHorizontal().x(d => d.y).y(d => d.x));

    const node = g.selectAll(".node")
        .data(nodes.descendants())
        .enter().append("g")
        .attr("class", "node")
        .attr("transform", d => "translate(" + d.y + "," + d.x + ")");

    node.append("circle")
        .attr("r", 10)
        .on("click", function(event, d) {{
            // ----------------------------------------------------
            // 1. 트리 하이라이트 및 기본 차트 그리기
            // ----------------------------------------------------
            d3.selectAll("circle").classed("selected", false);
            d3.select(this).classed("selected", true);
            
            const ancestors = new Set(d.ancestors());
            link.classed("highlight", l => ancestors.has(l.target))
                .classed("dimmed", l => !ancestors.has(l.target));

            // 메인 차트 그리기
            if (d.data.chart_config) {{
                document.getElementById("placeholder").style.display = "none";
                const config = d.data.chart_config;
                Plotly.newPlot('plotly-chart', config.data, config.layout, {{responsive: true}});
            }} else {{
                Plotly.purge('plotly-chart');
                document.getElementById("placeholder").style.display = "block";
            }}
            
            // 값 라벨 표시 (기존 로직)
            d3.selectAll(".value-label-group").remove();
            if (d.data.value !== undefined) {{
                const parentGroup = d3.select(this.parentNode);
                const labelGroup = parentGroup.append("g")
                    .attr("class", "value-label-group")
                    .attr("transform", "translate(0, -25)");
                
                labelGroup.append("rect")
                    .attr("class", "value-label-bg")
                    .attr("x", -25).attr("y", -10).attr("width", 50).attr("height", 20).attr("rx", 5);
                
                labelGroup.append("text")
                    .attr("class", "value-label-text")
                    .attr("x", 0).attr("y", 4).attr("text-anchor", "middle")
                    .text(d.data.value);
            }}

            // ----------------------------------------------------
            // 2. [신규 기능] 연관 센서 리스트업 (40개)
            // ----------------------------------------------------
            const listContent = document.getElementById("list-content");
            const listHeader = document.getElementById("list-header");
            listContent.innerHTML = ""; // 초기화

            if (d.data.related_sensors && d.data.related_sensors.length > 0) {{
                listHeader.innerText = `Related to: ${{d.data.name}} (${{d.data.related_sensors.length}})`;
                
                d.data.related_sensors.forEach(sensor => {{
                    const item = document.createElement("div");
                    item.className = "sensor-item";
                    item.innerHTML = `<span>${{sensor.name}}</span> <span class="sensor-val">${{sensor.value}}</span>`;
                    
                    // 연관 센서 클릭 이벤트
                    item.onclick = function() {{
                        // 리스트 아이템 하이라이트
                        document.querySelectorAll(".sensor-item").forEach(el => el.classList.remove("active"));
                        item.classList.add("active");

                        // JS에서 동적으로 차트 그리기 (Raw Data 사용)
                        drawClientSideChart(sensor.name, sensor.history);
                    }};
                    
                    listContent.appendChild(item);
                }});
            }} else {{
                listHeader.innerText = "Related Sensors";
                listContent.innerHTML = "<div style='color:#999; text-align:center; margin-top:20px;'>No related sensors</div>";
            }}

            event.stopPropagation();
        }});

    node.append("text")
        .attr("dy", ".35em")
        .attr("x", d => d.children ? -15 : 15)
        .style("text-anchor", d => d.children ? "end" : "start")
        .text(d => d.data.name);

    // JS에서 차트를 그리는 헬퍼 함수 (연관 센서용)
    function drawClientSideChart(name, yValues) {{
        const trace = {{
            y: yValues,
            mode: 'lines+markers',
            name: name,
            line: {{color: '#1565c0', width: 3, shape: 'spline'}}, // 연관 센서는 파란색 계열로 구분
            marker: {{size: 8, color: 'white', line: {{width: 2, color: '#1565c0'}}}}
        }};
        
        const layout = {{
            title: {{text: `${{name}} Trend`, font: {{size: 20, color: '#333'}}}},
            xaxis: {{title: 'Time Step', showgrid: true, gridcolor: '#eee'}},
            yaxis: {{title: 'Value', showgrid: true, gridcolor: '#eee'}},
            paper_bgcolor: 'rgba(0,0,0,0)',
            plot_bgcolor: 'rgba(0,0,0,0)',
            margin: {{l: 50, r: 30, t: 60, b: 50}},
            showlegend: false
        }};
        
        Plotly.newPlot('plotly-chart', [trace], layout, {{responsive: true}});
    }}

    window.resetView = function() {{
        link.classed("highlight", false).classed("dimmed", false);
        d3.selectAll("circle").classed("selected", false);
        d3.selectAll(".value-label-group").remove();
        
        // 리스트 초기화
        document.getElementById("list-header").innerText = "Related Sensors";
        document.getElementById("list-content").innerHTML = "<div style='color:#999; text-align:center; margin-top:20px;'>Select a node</div>";

        svg.transition().duration(750).call(
            zoom.transform, 
            d3.zoomIdentity.translate(80, height/2)
        );
        
        Plotly.purge('plotly-chart');
        document.getElementById("placeholder").style.display = "block";
    }}

    resetView();

</script>
</body>
</html>
"""

with open("tree_with_related_sensors.html", "w", encoding="utf-8") as f:
    f.write(html_content)

print("파일 생성 완료: tree_with_related_sensors.html")
print("노드를 클릭하면 중앙에 [연관 센서 리스트]가 나타나고, 리스트 항목을 클릭하면 [차트]가 갱신됩니다.")

파일 생성 완료: tree_with_related_sensors.html
노드를 클릭하면 중앙에 [연관 센서 리스트]가 나타나고, 리스트 항목을 클릭하면 [차트]가 갱신됩니다.


In [2]:
import pandas as pd
import numpy as np
import os

# 데이터 저장 폴더 생성
os.makedirs("data", exist_ok=True)

sensors = ['top temp', 'bottom temp', 'h2', 'nf3', 'pressure', 'throttle valve']

print("Parquet 파일 생성 중...")

for sensor in sensors:
    # 1. 시계열 데이터 생성 (행: 시간, 열: 센서값들)
    # 메인 센서 값 + 연관 센서 40개 값
    cols = {'value': np.random.randint(30, 80, 20)} # 메인 센서 (최근 20개 포인트)
    
    for i in range(1, 41):
        col_name = f"{sensor}_sub_{i:02d}"
        cols[col_name] = cols['value'] + np.random.randint(-5, 5, 20)
        
    df = pd.DataFrame(cols)
    
    # 2. Parquet 파일로 저장 (파일명: 센서이름.parquet)
    # 공백 등 특수문자 처리를 위해 파일명 안전하게 변환
    safe_name = sensor.replace(" ", "_")
    file_path = f"data/{safe_name}.parquet"
    
    df.to_parquet(file_path, engine='pyarrow')
    print(f" -> 생성 완료: {file_path}")

print("모든 Mock 데이터 생성 완료!")

Parquet 파일 생성 중...


ImportError: Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.